![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Similitud y normalización de textos

En este notebook aprenderá a calcular la similitud entre diferentes textos y a normalizarlos usando sklearn y [nltk](https://www.nltk.org/).

Este notebook tiene una licencia de [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). Un agradecimiento especial para [
Adrien sieg](https://medium.com/@adriensieg/text-similarities-da019229c894)

## Instrucciones Generales

La similitud y normalización de textos son tecnicas del procesamiento de lenguaje natural. Mientras que la similitud permite identificar que tan similares son un par de textos, la normalización permite convertir una palabra en su forma más básica.

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá a obtener la similitud entre dos textos usando diferentes métricas. En la segunda parte, normalizará el texto del set de noticias populares de UCI, eliminando stopwords y haciedo stemming y lematización. Para conocer más detalles de la base, puede ingresar al siguiente [vínculo](https://archive.ics.uci.edu/ml/datasets/online+news+popularity#).
   
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook.

In [2]:
import sys
print(sys.version)
import os
os.getcwd()

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]


'c:\\Users\\madelgado\\Documents\\GitHub\\MIAD_ML_NLP_2025\\Semana 5'

In [2]:
#pip install numpy==2.0.2 tensorflow==2.19.0 scikit-learn==1.6.1 scipy==1.13.1 nltk==3.9.1 pandas==2.2.3

In [3]:
#pip install -r requirements.txt 

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Similitud de texto

### Similitud de Jaccard
La similitud de Jaccard se define como el tamaño de la intersección dividido por el tamaño de la unión de dos conjuntos.

In [6]:
# Definición función de similitud de Jaccard que recibe como parámetros dos textos y retorna su similitud
def jaccard_similarity(query, document):
    # Calculo de la intersección
    intersection = set(query.split()).intersection(set(document.split()))
    # Calculo de la unión
    union = set(query.split()).union(set(document.split()))
    return len(intersection)/len(union)

In [50]:
# Definición de oraciones para calculo de similitud
s1 = "El preprocesamiento de texto es importante para los modelos de NLP"
s2 = "El NLP es importante para diferentes problemas en organizaciones"



In [49]:
# Impresión de la similitud de Jaccard entre las dos frases
jaccard_similarity(s1, s2)

0.16666666666666666

### Similitud de coseno

La similitud del coseno calcula la similitud midiendo el coseno del ángulo entre dos vectores.

In [16]:
# Importación librerías
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
import numpy as np

#### Similitud de coseno CountVectorizer
Al vectorizar con CountVectorizer, este tiene la limitación que palabras de un carácter no se consideran dentro del vocabulario, por ejemplo las palabras 'a' e 'y'. Con esto se tiene:

In [17]:
# Definición función de similitud de Coseno que recibe como parámetros dos textos y retorna su similitud
def cosine_distance_countVectorizer(s1, s2):

    # Uso de CountVectorizer para obtener vectores de una frase
    vect = CountVectorizer(binary=True)
    X_dtm = vect.fit_transform([s1, s2]).toarray()

    return 1-cosine(X_dtm[0],X_dtm[1])

In [51]:
# Impresión de la similitud de coseno entre las dos frases definidas anteriormente
cosine_distance_countVectorizer(s1, s2)

np.float64(0.5270462766947299)

#### Similitud de coseno manual
Al realizar la creación los vectores de la frase manualmente se garantiza que se consideran todas las palabras. Con esto:

In [19]:
def obtener_vectores(union, s1, s2):

    s1_l = []
    s2_l = []

    for palabra in union:
        if palabra in s1.split():
            s1_l.append(1)
        else:
            s1_l.append(0)

        if palabra in s2.split():
            s2_l.append(1)
        else:
            s2_l.append(0)

    return s1_l, s2_l

# Definición función de similitud de Coseno que recibe como parámetros dos textos y retorna su similitud
def cosine_distance_manual(s1, s2):

    union = list(set(s1.split()).union(set(s2.split())))

    s1_v, s2_v = obtener_vectores(union, s1, s2)

    return 1-cosine(s1_v, s2_v)

In [20]:
# Impresión de la similitud de coseno entre las dos frases definidas anteriormente
cosine_distance_manual(s1, s2)

np.float64(0.27386127875258304)

La diferencia entre las distancias de coseno se obtiene por la forma de vectorizar, consideren esta segunda para el desarrollo del quiz. Los invitamos a que entiendan con detalle que hace la función manual.

### Codificación de Oraciones y Similitud de Coseno

La codificación de oraciones es una de las representaciones más populares del vocabulario de documentos. Es capaz de capturar el contexto de una palabra en un documento, la similitud semántica y sintáctica, la relación con otras palabras, etc.

Para esta sección del notebook instale la libreria tensorflow y tensorflow_hub (si aun no las ha instalado) con el comando *!pip install tensorflow* y *!pip install tensorflow_hub* respectivamente.

In [21]:
# Importación librerías
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub

In [22]:
## Importación el módulo TF Hub del Universal Sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)

In [24]:
# Codificación de las frases anteriormente definidas con la libreria tensorflow
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed([s1, s2]))

In [25]:
#Impresión de las codificaciones
sentences_embeddings

array([[ 0.00112492, -0.05461062, -0.05706681, ..., -0.07854681,
        -0.0074173 , -0.08419687],
       [-0.00924624,  0.03860932, -0.04020129, ..., -0.08117636,
        -0.05272641, -0.09135485]], dtype=float32)

In [26]:
# Impresión de la similitud de coseno entre las dos frases definidas anteriormente usando codificación de oraciones
1-cosine(sentences_embeddings[0], sentences_embeddings[1])

np.float32(0.7092906)

## Normalización de textos

In [27]:
# Importación librerías
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [28]:
# Carga de datos de archivos .csv
df = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/mashable_texts.csv', index_col=0)
df.head()

,author,author_web,shares,text,title,facebo,google,linked,twitte,twitter_followers
0,Seth Fiegerman,http://mashable.com/people/seth-fiegerman/,4900,\nApple's long and controversial ebook case ha...,The Supreme Court smacked down Apple today,http://www.facebook.com/sfiegerman,NaN,http://www.linkedin.com/in/sfiegerman,https://twitter.com/sfiegerman,14300
1,Rebecca Ruiz,http://mashable.com/people/rebecca-ruiz/,1900,Analysis\n\n\n\n\n\nThere is a reason that Don...,Every woman has met a man like Donald Trump,NaN,NaN,NaN,https://twitter.com/rebecca_ruiz,3738
2,Davina Merchant,http://mashable.com/people/568bdab351984019310...,7000,LONDON - Last month we reported on a dog-sized...,Adorable dog-sized rabbit finally finds his fo...,NaN,https://plus.google.com/105525238342980116477?...,NaN,NaN,0
3,Scott Gerber,[],5000,Today's digital marketing experts must have a ...,15 essential skills all digital marketing hire...,NaN,NaN,NaN,NaN,0
4,Josh Dickey,http://mashable.com/people/joshdickey/,1600,"LOS ANGELES — For big, fun, populist popcorn m...",Mashable top 10: 'The Force Awakens' is the be...,NaN,https://plus.google.com/109213469090692520544?...,NaN,https://twitter.com/JLDlite,11200


In [29]:
# Separación de variable de interés (y)
y = df.shares
y.describe()

count       82.000000
mean      3090.487805
std       8782.031594
min        437.000000
25%        893.500000
50%       1200.000000
75%       2275.000000
max      63100.000000
Name: shares, dtype: float64

In [30]:
# Categoización de la variable de interés (y)
y = pd.cut(y, [0, 893, 1200, 2275, 63200], labels=[0, 1, 2, 3])
y.value_counts()

shares
1    22
0    21
3    21
2    18
Name: count, dtype: int64

In [31]:
# Definición de variable de interés en el dataframe
df['y'] = y

In [32]:
# Definición de variables predictoras
X = df.text

In [33]:
# Definición de función que recibe un texto vectorizado y calcula el acurracy de un modelo Naive Bayes
def tokenize_test(vect):
    X_dtm = vect.fit_transform(X)
    print('Features: ', X_dtm.shape[1])
    nb = MultinomialNB()
    print(pd.Series(cross_val_score(nb, X_dtm, y, cv=10)).describe())

### Eliminación de stopwords

In [34]:
# Eliminación de stopwords al usar el parámetro 'stop_words' de la función CountVectorizer()
vect_no_stopw = CountVectorizer(stop_words='english')

In [35]:
# Impresión de stopwords del texto
print(vect_no_stopw.get_stop_words())

frozenset({'on', 'whereby', 'thick', 'amongst', 'itself', 'since', 'an', 'how', 'cannot', 'is', 'where', 'move', 'cry', 'hers', 'rather', 'although', 'please', 'whom', 'hereafter', 'already', 'serious', 'few', 'least', 'them', 'fifteen', 'his', 'whoever', 'both', 'whence', 'all', 'much', 'anyway', 'thereupon', 'back', 'everything', 'it', 'their', 'ie', 'nowhere', 'fire', 'anything', 'there', 'your', 'describe', 'i', 'nevertheless', 'now', 'however', 'only', 'that', 'by', 'one', 'are', 'not', 'among', 'onto', 'she', 'made', 'couldnt', 'am', 'thereby', 'what', 'too', 'became', 'hence', 'without', 'those', 'who', 'beside', 'another', 'put', 'per', 'anywhere', 'anyhow', 'were', 'four', 'about', 'each', 'above', 'may', 'almost', 'he', 'as', 'sixty', 'most', 'others', 'none', 'we', 'along', 'show', 'thereafter', 'out', 'yourself', 'beyond', 'anyone', 'bill', 'everyone', 'less', 'become', 'find', 'do', 'someone', 'been', 'ours', 'while', 'her', 'very', 'down', 'have', 'sincere', 'first', 'det

In [36]:
# Desempeño del modelo sin considerar stopwords
tokenize_test(vect_no_stopw)

Features:  7710
count    10.000000
mean      0.331944
std       0.146614
min       0.111111
25%       0.250000
50%       0.354167
75%       0.468750
max       0.500000
dtype: float64


### Stemming

Stemming es un preprocesamiento del texto en el que para cada palabra se obtiene su raíz o en inglés stem.

In [37]:
# Inicialización de stemmer
stemmer = SnowballStemmer('english')

In [38]:
# Creación de matrices de documentos usando CountVectorizer a partir de X
vect = CountVectorizer()
vect.fit(X)

CountVectorizer()

In [40]:
# Definiicón de lista con vocabulario de la matriz de documentos
words = list(vect.vocabulary_.keys())[:100]
words

['apple',
 'long',
 'and',
 'controversial',
 'ebook',
 'case',
 'has',
 'reached',
 'its',
 'final',
 'chapter',
 'it',
 'not',
 'the',
 'happy',
 'ending',
 'company',
 'wanted',
 'supreme',
 'court',
 'on',
 'monday',
 'rejected',
 'an',
 'appeal',
 'filed',
 'by',
 'to',
 'overturn',
 'stinging',
 'ruling',
 'that',
 'led',
 'broad',
 'conspiracy',
 'with',
 'several',
 'major',
 'publishers',
 'fix',
 'price',
 'of',
 'books',
 'sold',
 'through',
 'online',
 'bookstore',
 'decision',
 'means',
 'now',
 'no',
 'choice',
 'but',
 'pay',
 'out',
 '400',
 'million',
 'consumers',
 'additional',
 '50',
 'in',
 'legal',
 'fees',
 'according',
 'original',
 'settlement',
 '2014',
 'see',
 'also',
 'here',
 'how',
 'marshalled',
 'entire',
 'tech',
 'industry',
 'fight',
 'fbi',
 'for',
 'verdict',
 'is',
 'more',
 'damaging',
 'reputation',
 'as',
 'consumer',
 'friendly',
 'brand',
 'mention',
 'legacy',
 'beloved',
 'founder',
 'steve',
 'jobs',
 'than',
 'actual',
 'bottom',
 'line',

In [41]:
# Obtención e impresión de los stem de cada palabra de la lista
print([stemmer.stem(word) for word in words])

['appl', 'long', 'and', 'controversi', 'ebook', 'case', 'has', 'reach', 'it', 'final', 'chapter', 'it', 'not', 'the', 'happi', 'end', 'compani', 'want', 'suprem', 'court', 'on', 'monday', 'reject', 'an', 'appeal', 'file', 'by', 'to', 'overturn', 'sting', 'rule', 'that', 'led', 'broad', 'conspiraci', 'with', 'sever', 'major', 'publish', 'fix', 'price', 'of', 'book', 'sold', 'through', 'onlin', 'bookstor', 'decis', 'mean', 'now', 'no', 'choic', 'but', 'pay', 'out', '400', 'million', 'consum', 'addit', '50', 'in', 'legal', 'fee', 'accord', 'origin', 'settlement', '2014', 'see', 'also', 'here', 'how', 'marshal', 'entir', 'tech', 'industri', 'fight', 'fbi', 'for', 'verdict', 'is', 'more', 'damag', 'reput', 'as', 'consum', 'friend', 'brand', 'mention', 'legaci', 'belov', 'founder', 'steve', 'job', 'than', 'actual', 'bottom', 'line', 'put', 'fine', 'context']


### Lematización

La lemmatización es un proceso en el que se busca el lema de cada palabra de un texto, siendo un lema la forma base o de diccionario de una palabra.

In [42]:
# Importación de librerias
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\madelgado\AppData\Roaming\nltk_data...


True

In [43]:
# Obtención e impresión de los lemas de cada palabra de la lista asumiendo que cada palabra es un sustantivo
print([wordnet_lemmatizer.lemmatize(word) for word in words])

['apple', 'long', 'and', 'controversial', 'ebook', 'case', 'ha', 'reached', 'it', 'final', 'chapter', 'it', 'not', 'the', 'happy', 'ending', 'company', 'wanted', 'supreme', 'court', 'on', 'monday', 'rejected', 'an', 'appeal', 'filed', 'by', 'to', 'overturn', 'stinging', 'ruling', 'that', 'led', 'broad', 'conspiracy', 'with', 'several', 'major', 'publisher', 'fix', 'price', 'of', 'book', 'sold', 'through', 'online', 'bookstore', 'decision', 'mean', 'now', 'no', 'choice', 'but', 'pay', 'out', '400', 'million', 'consumer', 'additional', '50', 'in', 'legal', 'fee', 'according', 'original', 'settlement', '2014', 'see', 'also', 'here', 'how', 'marshalled', 'entire', 'tech', 'industry', 'fight', 'fbi', 'for', 'verdict', 'is', 'more', 'damaging', 'reputation', 'a', 'consumer', 'friendly', 'brand', 'mention', 'legacy', 'beloved', 'founder', 'steve', 'job', 'than', 'actual', 'bottom', 'line', 'put', 'fine', 'context']


In [44]:
# Obtención e impresión de los lemas de cada palabra de la lista asumiendo que cada palabra es un verbo
print([wordnet_lemmatizer.lemmatize(word,pos='v') for word in words])

['apple', 'long', 'and', 'controversial', 'ebook', 'case', 'have', 'reach', 'its', 'final', 'chapter', 'it', 'not', 'the', 'happy', 'end', 'company', 'want', 'supreme', 'court', 'on', 'monday', 'reject', 'an', 'appeal', 'file', 'by', 'to', 'overturn', 'sting', 'rule', 'that', 'lead', 'broad', 'conspiracy', 'with', 'several', 'major', 'publishers', 'fix', 'price', 'of', 'book', 'sell', 'through', 'online', 'bookstore', 'decision', 'mean', 'now', 'no', 'choice', 'but', 'pay', 'out', '400', 'million', 'consumers', 'additional', '50', 'in', 'legal', 'fee', 'accord', 'original', 'settlement', '2014', 'see', 'also', 'here', 'how', 'marshal', 'entire', 'tech', 'industry', 'fight', 'fbi', 'for', 'verdict', 'be', 'more', 'damage', 'reputation', 'as', 'consumer', 'friendly', 'brand', 'mention', 'legacy', 'beloved', 'founder', 'steve', 'job', 'than', 'actual', 'bottom', 'line', 'put', 'fine', 'context']


In [45]:
# Definición de la función que tenga como parámetro texto y devuelva una lista de lemas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

In [46]:
# Creación de matrices de documentos usando CountVectorizer, usando el parámetro 'split_into_lemmas'
vect_lemas = CountVectorizer(analyzer=split_into_lemmas)

In [47]:
# Desempeño del modelo al lematizar el texto
tokenize_test(vect_lemas)

Features:  10208
count    10.000000
mean      0.379167
std       0.108584
min       0.222222
25%       0.281250
50%       0.375000
75%       0.486111
max       0.500000
dtype: float64
